<a href="https://colab.research.google.com/github/juseonmin/bioinfo/blob/main/Project_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 생물정보학 및 실습 1 - 실습 프로젝트 : Figure 2E 구현하기

```
생물정보학 및 실습 1

2022-22456 주선민 

실습 프로젝트

```

## Google Drive 마운트

drive에 연결하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

## 실습용 Bioconda 환경 설치하기

In [ ]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

fatal: destination path 'colab-biolab' already exists and is not an empty directory.


In [ ]:
!conda install -y bedtools bioawk

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bioawk-1.0                 |       h7132678_7         196 KB  bioconda
    ------------------------------------------------------------
                                           Total:         196 KB

The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7



bioawk-1.0           | 196 KB    | : 100% 1.0/1 [00:00<00:00,  2.36it/s]                
Preparing transaction: \ done
Verify

## Term Project
논문의 **Figure 2E**를 재현하기 위한 코드 작성입니다.


(자외선에 의해서 단백질이 붙어있다가 단백질 분해효소로 단백질을 뜯어내고 남은 자리에서는 reversetranscriptase가 오류를 자주 냅니다. 오류율을 분석해서 단백질이 앉아 있었던 자리를 분석해 봅시다.)

In [ ]:
%cd /content/drive/MyDrive/binfo1-work/

/content/drive/MyDrive/binfo1-work


mRNA binding motif를 조사하기 위해서 genome 전체 단위로 범위를 넓혀 pileup을 진행하였습니다. 그리고 파일 크기를 가늠하기 위해 라인 수를 세주었습니다.

In [ ]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
!wc -l CLIP-35L33G.pileup

pileup 파일을 열어 한 번 확인하였습니다.

In [ ]:
!head CLIP-35L33G.pileup

아무래도 전체 read 상대로 pileup 파일을 만들다보니 그 용량이 매우 커 read depth cut 기준인 50이 넘는 count 를 가진 파일만 불러 왔습니다

**현재 base 정보에는 mapping quality를 나타내는 문자와 끝을 알리는 문자 등등 여러 문자가 섞여 있기 때문에 먼저 적당량의 파일을 불러들인 후 다시 한번 cut 할 예정입니다.**

In [ ]:
!awk '$4 >= 50 { print $0; }' CLIP-35L33G.pileup > CLIP-35L33G-readcut.pileup
!head CLIP-35L33G-readcut.pileup

이제 pandas에 불러오겠습니다.

In [ ]:
import pandas as pd

pileup= pd.read_csv('CLIP-35L33G-readcut.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup


,chrom,pos,_ref,count,basereads,quals
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...
...,...,...,...,...,...,...
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...


match와 substitution에 해당하는 tag 이외에 제거하기 위한 단계입니다.
read의 시작에 붙는 ^(ascii code)도 없애주었습니다.

In [ ]:
pileup['matches']=pileup['basereads'].str.replace(pat=r'[<>$*#]+', repl= r'', regex=True)
pileup['matches']=pileup['matches'].str.replace(pat='\^(.)', repl= r'', regex=True) 
pileup

,chrom,pos,_ref,count,basereads,quals,matches
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...,CCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...,...,...
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


아래 정보 처리량을 조금 줄이고자 특수 문자를 지운 상태에서 read depth가 50이 넘는 것으로 추려보았습니다.

In [ ]:
pileup_readcut = pileup[(pileup['matches'].str.len() >= 50)]
pileup_readcut

,chrom,pos,_ref,count,basereads,quals,matches
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...,CCCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...,...,...,...,...
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [ ]:
df_length = pileup_readcut[['chrom','pos','matches']]

entropy를 구하기 전 간단하게 처리해주었습니다.

In [ ]:
for i in 'A','T','C','G':
  df_length[i] = pileup_readcut['matches'].str.count(i)

In [ ]:
df_length['length'] = df_length[['A','T','C','G']].sum(axis=1)
df_length

### Crosslinking-induced reverse-transcription error score (CRES) 즉, Shannon Entropy 구하기

In [ ]:
import numpy as np
new_table = pd.DataFrame()
for i in 'A','T','C','G':
    new_table[i] = np.log2(1/(df_length[i]/df_length['length']))*(df_length[i]/df_length['length'])
new_table



In [ ]:
df_length['entropy'] = new_table.sum(axis=1)
df_length 

In [ ]:
df_entropy = df_length[(df_length['entropy'] >= 0.8)]
df_entropy

In [ ]:
df_entropy['pos_b20'] = df_entropy['pos'].apply(lambda x : x-20) 
df_entropy['pos_f20'] = df_entropy['pos'].apply(lambda x : x+21)
df_entropy

In [ ]:
df_entropy.to_csv('entropy_0603.txt', sep = ' ', index=None)

In [ ]:
bedfile = df_entropy[['chrom','pos_b20','pos_f20','pos']]
bedfile.to_csv('entropy.bed', sep = ' ', header=None, index=None)

In [ ]:
!samtools view CLIP-33L33G -L entropy_1.bed | less

In [ ]:
!samtools fasta CLIP-35L33G.bam > CLIP-35L33G.fa

[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 24168325 reads


In [ ]:
!samtools help fasta

Usage: samtools fasta [options...] <in.bam>

Description:
Converts a SAM, BAM or CRAM to FASTA format.

Options:
  -0 FILE      write reads designated READ_OTHER to FILE
  -1 FILE      write reads designated READ1 to FILE
  -2 FILE      write reads designated READ2 to FILE
  -o FILE      write reads designated READ1 or READ2 to FILE
               note: if a singleton file is specified with -s, only
               paired reads will be written to the -1 and -2 files.
  -f INT       only include reads with all  of the FLAGs in INT present [0]
  -F INT       only include reads with none of the FLAGS in INT present [0x900]
  -G INT       only EXCLUDE reads with all  of the FLAGs in INT present [0]
  -n           don't append /1 and /2 to the read name
  -N           always append /1 and /2 to the read name
  -s FILE      write singleton reads designated READ1 or READ2 to FILE
  -t           copy RG, BC and QT tags to the FASTA header line
  -T TAGLIST   copy arbitrary tags to the FASTA hea

In [ ]:
!samtools fasta -G 16 -t CLIP-35L33G.bam


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
>SRR458758.26312664
AATCAGTGGTGGAAGAACGGTCTCAGAACTGTTTGTCTCAATT
>SRR458758.26340098
AATCAGTGGTGGAACAACGGTCTCAGAACTGTTTGTCTCAATT
>SRR458758.26351524
AATCAGTGGTGGAAAACGGTCTCAGAACTGTTTGTCTCAATTGGC
>SRR458758.26355114
AATCAGTGGTGGAACAACGGTCTCAGAACTGTTTGTCTCAATTGGC
>SRR458758.26357229
AATCAGTGGTGGAACAACGGTCTCAGAACTGTTTGTCTCAATTG
>SRR458758.26441269
AATCAGTGGTGGAAGAACGGTCT
>SRR458758.26446436
AATCAGTGGTGGAAGAACGGTCTCAGAACTGT
>SRR458758.26542216
AATCAGTGGTGGAAGAACGGT
>SRR458758.26633872
AATCAGTGGTGGAAGAACGGTCTCAGAACTGTTTGTCTCAATTGGC
>SRR458758.26674024
AATCAGTGGTGGATAACGGTCTCAGAACTGTTTGTCTCAATTGGC
>SRR458758.26726238
AATCAGTGGTGGAAAACGGTCTCAGAACTGTTTGTCTCAATTGGC
>SRR458758.26733352
AATCAGTGGTGGAAAAACGGTCTCAGAACTGTTTGTCTCAAT
>SRR458758.26787506
AATCAGTGGTGGAAAAACGGTCTCAGAACTGTTTGTCTCAAT
>SRR458758.26800996
AATCAGTGGTGGAAGAACAGTCTCAGAACTGTTTGT
>SRR458758.26813503
AATCAGTGGTGAAAGAACGGTCTCAGAACTGTTTGTCT
>SRR458758.26856585
AATCAGTGGTGGAAGAACGGTCT
>SRR458758.270

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TGCTGGGATTTGAAGTCAGGATCT
>SRR458758.27667797
AGCTACCATGTGGTTGCTGGGATTTGAAGTCAGGATCT
>SRR458758.27918100
AGCTACCATGTGGTTGCTGGGATTTGAAGTCAGGATCT
>SRR458758.8400860
AAAGAAGATAAGATTAAGGCC
>SRR458758.11448381
AAAGAAGATAAGATTAAGGCC
>SRR458758.15781670
AAAGAAGATAAGATTAAGGCC
>SRR458758.24324956
AAAGAAGATAAGATTAAGGCC
>SRR458758.27971280
AAAGAAGATAAGATTAAGGCC
>SRR458758.14549244
CTGCCAGTTCAACAACATAATACTAGGCC
>SRR458758.26912898
CTGCCAGTTCAACAACATAATACTAGGCC
>SRR458758.27313460
CTGCCAGTTCAACAACATAATACTAGGCC
>SRR458758.15199389
TTCTTTCCTTCTTTCCTTCTTTCCTTCTTTCCTTCTTTCCTTCTTTC
>SRR458758.15844475
TTCTTTCCTTCTTTCCTTCTTTCCTTCTTTCCTTCTTTCCTTCTTTC
>SRR458758.2457047
ATGAACCTCGTAGATGATGCAGAAGAGATTCATTCT
>SRR458758.2467249
ATGAACCTCGTAGATGATGCAGAAGAGATTCATTCT
>SRR458758.4062252
ATGAACCTCGTTAGATGATGCAGAAGAGATTCATTCT
>SRR458758.4275926
ATGAACCTCGTTAGATGATGCAGAAGAGATTCATTCT
>SRR458758.4743964
ATGAACCTCGTAGATGATGCAGAAGAGATTCATTCT
>SRR458758.5281287
ATGAACCTCGTTAGATGATGCAGAAGAGATTCATTCT
>SRR458758.7917818
ATGA

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
>SRR458758.29357308
ATTATGCTCTTAACGTGCATTTGAGCTGTGAAAAT
>SRR458758.29461853
ATTATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.30505695
ATTATGCTCTTAACGTGCATTTGAGCTGTGAAAAT
>SRR458758.31053070
ATTATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.5358913
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.5639486
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.8400712
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.10126752
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.13706919
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.23018330
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.27216076
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.27489068
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.28006882
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.28070676
ATGCTCTTAACATGCATTTGAGCTGTGAAA
>SRR458758.8295342
AACGTGCATTTGAGCTGTAAAAAT
>SRR458758.13594560
AACGTGCATTTGAGCTGTAAAAAT
>SRR458758.17277435
AACGTGCATTTGAGCTGTAAAAAT
>SRR458758.20576532
AACGTGCATTTGAGCTGTAAAAAT
>SRR458758.23933426
AACGTGCATTTGAGCTGTAAAAAT
>SRR458758.25

In [ ]:
!bedtools getfasta -fi CLIP-35L33G.fa -bed entropy.bed -name -fo CLIP-35L33G.fa.out

index file CLIP-35L33G.fa.fai not found, generating...
tcmalloc: large alloc 1073741824 bytes == 0x55b8c2066000 @  0x7f8b11100887 0x55b7af851487 0x55b7af970236 0x55b7af970609 0x55b7af97412a 0x55b7af886a8d 0x55b7af887dc8 0x55b7af885729 0x55b7af846a3a 0x7f8b1058ac87 0x55b7af84dee9
tcmalloc: large alloc 1207959552 bytes == 0x55b995202000 @  0x7f8b11100887 0x55b7af96e4d5 0x55b7af973bc2 0x55b7af973dfc 0x55b7af974176 0x55b7af886a8d 0x55b7af887dc8 0x55b7af885729 0x55b7af846a3a 0x7f8b1058ac87 0x55b7af84dee9
^C


In [ ]:
!cat CLIP-35L33G.fa.out